In [1]:
import pandas as pd

# Load the datasets
applications_df = pd.read_csv('cleaned_applications.csv')
branches_df = pd.read_csv('cleaned_branches.csv')
customers_df = pd.read_csv('cleaned_customers.csv')
defaults_df = pd.read_csv('cleaned_defaults.csv')
loans_df = pd.read_csv('cleaned_loans.csv')
transactions_df = pd.read_csv('cleaned_transactions.csv')

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [2]:
# Merge transactions and loans dataframes, ensuring Customer_ID is included
merged_df = pd.merge(transactions_df, loans_df[['Loan_ID', 'Customer_ID']], on='Loan_ID', how='left')

# Rename Customer_ID_y to Customer_ID
merged_df.rename(columns={'Customer_ID_y': 'Customer_ID'}, inplace=True)

# Determine if a loan was ever defaulted
merged_df['Defaulted'] = merged_df['Loan_ID'].isin(defaults_df['Loan_ID'])

# Group by Customer_ID and calculate the percentage of defaulted loans
customer_default_percentage = merged_df.groupby('Customer_ID')['Defaulted'].mean().reset_index()
customer_default_percentage.rename(columns={'Defaulted': 'Default_Percentage'}, inplace=True)

# Categorize customers based on default percentage
def categorize_repayment(percentage):
    if percentage == 0:
        return 'Always on Time'
    elif percentage > 0 and percentage <= 0.2:
        return 'Occasional Defaulter'
    else:
        return 'Frequent Defaulter'

customer_default_percentage['Repayment_Behavior'] = customer_default_percentage['Default_Percentage'].apply(categorize_repayment)

# Merge repayment behavior categories into customers_df
customers_df = pd.merge(customers_df, customer_default_percentage[['Customer_ID', 'Repayment_Behavior']], on='Customer_ID', how='left')

# Fill NaN values in Repayment_Behavior for customers with no transactions (assuming they are 'Always on Time')
customers_df['Repayment_Behavior'].fillna('Always on Time', inplace=True)

display(customers_df.head())

/var/folders/7v/b4cx31gj2m15lfjrqk3sy1240000gp/T/ipykernel_9204/3087054741.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers_df['Repayment_Behavior'].fillna('Always on Time', inplace=True)


Customer_ID      Full_Name  Contact_Number  \
0     C000001   Shayak Kumar      1549750578   
1     C000002    Divij Ghosh      2121540709   
2     C000003  Rhea Bhargava    910320534937   
3     C000004  Indrans Walia      6566372162   
4     C000005    Saanvi Chad      9281095585   

                              Email                                  Address  \
0  saxenaindrans@gopal-krishnan.com  27/062\nSangha Nagar, Bongaigaon 736150   
1           ramanindranil@tella.com   H.No. 20, Edwin Circle\nKatihar 892167   
2             bahlelakshi@mall.info       67/386\nComar Nagar, Chapra 137669   
3                akadakia@yahoo.com      88\nDhillon Ganj\nJamshedpur-816278   
4               arhaan11@master.net        011, Kunda Marg\nHyderabad-505161   

   Age  Gender Marital_Status Employment_Status  Annual_Income  Credit_Score  \
0   53    Male         Single        Unemployed         886810           433   
1   62  Female         Single     Self-Employed         396746           418   
2   62   Other         Single     Self-Employed        1118591           736   
3   59  Female         Single          Salaried         524075           353   
4   56    Male         Single     Self-Employed        1219355           735   

      Region  Account_Tenure Relationship_Manager_ID Repayment_Behavior  
0      North              12                  RM0015     Always on Time  
1       West              10                  RM0276     Always on Time  
2  Northeast               9                  RM0098     Always on Time  
3    Central              11                  RM0231     Always on Time  
4       West               1                  RM0107     Always on Time

In [3]:
# 1. Merge applications_df and customers_df on the 'Customer_ID' column.
applications_customers_df = pd.merge(applications_df, customers_df, on='Customer_ID', how='left')

# 2. Analyze the relationship between Approval_Status and demographic columns.
demographic_cols = ['Gender', 'Marital_Status', 'Education', 'Employment_Status']
approval_status_analysis = {}

In [4]:
# 2. Analyze the relationship between Approval_Status and available demographic columns.
demographic_cols = ['Gender', 'Marital_Status', 'Employment_Status']
approval_status_analysis = {}

for col in demographic_cols:
    approval_status_analysis[col] = applications_customers_df.groupby([col, 'Approval_Status']).size().unstack(fill_value=0)

# 3. Analyze the relationship between Rejection_Reason and available demographic columns.
rejection_reason_analysis = {}

for col in demographic_cols:
    rejection_reason_analysis[col] = applications_customers_df.groupby([col, 'Rejection_Reason']).size().unstack(fill_value=0)

# 4. Print or display the results.
print("Analysis of Approval Status vs. Demographics:")
for col, analysis in approval_status_analysis.items():
    print(f"\n{col}:\n")
    display(analysis)

print("\nAnalysis of Rejection Reason vs. Demographics:")
for col, analysis in rejection_reason_analysis.items():
    print(f"\n{col}:\n")
    display(analysis)

Analysis of Approval Status vs. Demographics:

Gender:



Approval_Status  Approved  Rejected
Gender                             
Female              23171      4180
Male                23448      4201
Other               23381      4219


Marital_Status:



Approval_Status  Approved  Rejected
Marital_Status                     
Divorced            23499      4248
Married             23334      4191
Single              23167      4161


Employment_Status:



...

Rejection_Reason  Incomplete Documents  Insufficient Income  Low Credit Score  \
Gender                                                                          
Female                            1421                 1336              1423   
Male                              1404                 1371              1426   
Other                             1379                 1392              1448   

Rejection_Reason  Not Applicable  
Gender                            
Female                     23171  
Male                       23448  
Other                      23381


Marital_Status:



Rejection_Reason  Incomplete Documents  Insufficient Income  Low Credit Score  \
Marital_Status                                                                  
Divorced                          1411                 1388              1449   
Married                           1402                 1365              1424   
Single                            1391                 1346              1424   

Rejection_Reason  Not Applicable  
Marital_Status                    
Divorced                   23499  
Married                    23334  
Single                     23167


Employment_Status:



Rejection_Reason   Incomplete Documents  Insufficient Income  \
Employment_Status                                              
Salaried                           1422                 1330   
Self-Employed                      1338                 1365   
Unemployed                         1444                 1404   

Rejection_Reason   Low Credit Score  Not Applicable  
Employment_Status                                    
Salaried                       1378           23452  
Self-Employed                  1443           23346  
Unemployed                     1476           23202

In [5]:
# 2. Calculate the total loan amount and number of loans for each customer using the loans_df.
customer_loan_metrics = loans_df.groupby('Customer_ID').agg(
    Total_Loan_Amount=('Loan_Amount', 'sum'),
    Number_of_Loans=('Loan_ID', 'count')
).reset_index()

# 3. Merge the calculated loan metrics with the customers_df (which now includes 'Repayment_Behavior').
customers_with_loan_metrics = pd.merge(customers_df, customer_loan_metrics, on='Customer_ID', how='left')

# Fill NaN values for customers with no loans
customers_with_loan_metrics['Total_Loan_Amount'].fillna(0, inplace=True)
customers_with_loan_metrics['Number_of_Loans'].fillna(0, inplace=True)


# 1. Define criteria for "high-value customers".
# Criteria:
# - Always on Time repayment behavior
# - Total Loan Amount in the top 25th percentile
# - Number of Loans in the top 25th percentile

high_value_loan_amount_threshold = customers_with_loan_metrics['Total_Loan_Amount'].quantile(0.75)
high_value_number_of_loans_threshold = customers_with_loan_metrics['Number_of_Loans'].quantile(0.75)


# 4. Filter the merged DataFrame to identify customers who meet the defined "high-value" criteria.
high_value_customers_df = customers_with_loan_metrics[
    (customers_with_loan_metrics['Repayment_Behavior'] == 'Always on Time') &
    (customers_with_loan_metrics['Total_Loan_Amount'] >= high_value_loan_amount_threshold) &
    (customers_with_loan_metrics['Number_of_Loans'] >= high_value_number_of_loans_threshold)
]

# 5. Display the information for the identified high-value customers.
display(high_value_customers_df[['Customer_ID', 'Total_Loan_Amount', 'Number_of_Loans', 'Repayment_Behavior']].head())

/var/folders/7v/b4cx31gj2m15lfjrqk3sy1240000gp/T/ipykernel_9204/4117782844.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers_with_loan_metrics['Total_Loan_Amount'].fillna(0, inplace=True)
/var/folders/7v/b4cx31gj2m15lfjrqk3sy1240000gp/T/ipykernel_9204/4117782844.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we

Customer_ID  Total_Loan_Amount  Number_of_Loans Repayment_Behavior
2      C000003          6194007.0              2.0     Always on Time
3      C000004         11159729.0              3.0     Always on Time
6      C000007          7678817.0              2.0     Always on Time
12     C000013         19829940.0              6.0     Always on Time
16     C000017          6863821.0              2.0     Always on Time

In [6]:
# 1. Summarize repayment behavior categories
repayment_distribution = customers_df['Repayment_Behavior'].value_counts(normalize=True) * 100
print("Distribution of Customers by Repayment Behavior:")
print(repayment_distribution)

# 2. Summarize loan approval/rejection patterns by demographics
print("\nSummary of Loan Approval Status by Demographics:")
for col, analysis in approval_status_analysis.items():
    print(f"\n{col}:\n")
    print(analysis)

print("\nSummary of Loan Rejection Reasons by Demographics:")
for col, analysis in rejection_reason_analysis.items():
    print(f"\n{col}:\n")
    print(analysis)

# 3. Describe high-value customers and their count
print("\nHigh-Value Customers:")
print(f"Criteria for high-value customers: 'Always on Time' repayment behavior AND Total Loan Amount >= {high_value_loan_amount_threshold:.2f} AND Number of Loans >= {high_value_number_of_loans_threshold:.2f}")
print(f"Total number of high-value customers identified: {len(high_value_customers_df)}")
print("\nExamples of High-Value Customers:")
display(high_value_customers_df.head())

Distribution of Customers by Repayment Behavior:
Repayment_Behavior
Always on Time          88.570000
Frequent Defaulter      10.478571
Occasional Defaulter     0.951429
Name: proportion, dtype: float64

Summary of Loan Approval Status by Demographics:

Gender:

Approval_Status  Approved  Rejected
Gender                             
Female              23171      4180
Male                23448      4201
Other               23381      4219

Marital_Status:

Approval_Status  Approved  Rejected
Marital_Status                     
Divorced            23499      4248
Married             23334      4191
Single              23167      4161

Employment_Status:

Approval_Status    Approved  Rejected
Employment_Status                    
Salaried              23452      4130
Self-Employed         23346      4146
Unemployed            23202      4324

Summary of Loan Rejection Reasons by Demographics:

Gender:

Rejection_Reason  Incomplete Documents  Insufficient Income  Low Credit Score  \
Gende

Customer_ID      Full_Name  Contact_Number                   Email  \
2      C000003  Rhea Bhargava    910320534937   bahlelakshi@mall.info   
3      C000004  Indrans Walia      6566372162      akadakia@yahoo.com   
6      C000007   Dishani Dube      6144096314         ngala@yahoo.com   
12     C000013   Ahana  Raman    914114393937  raunakvala@hotmail.com   
16     C000017   Purab Thaker      5962588263    arnavsom@hotmail.com   

                                   Address  Age  Gender Marital_Status  \
2       67/386\nComar Nagar, Chapra 137669   62   Other         Single   
3      88\nDhillon Ganj\nJamshedpur-816278   59  Female         Single   
6   03/41\nBadami Circle\nAlappuzha 624834   39    Male       Divorced   
12     18/75, Setty Path, Ghaziabad 701570   60    Male       Divorced   
16          77, Gaba Nagar, Aligarh 955025   60    Male        Married   

   Employment_Status  Annual_Income  Credit_Score     Region  Account_Tenure  \
2      Self-Employed        1118591           736  Northeast               9   
3           Salaried         524075           353    Central              11   
6      Self-Employed        1237215           548    Central               5   
12     Self-Employed        1245092           460      North               5   
16     Self-Employed        1776242           757      South               8   

   Relationship_Manager_ID Repayment_Behavior  Total_Loan_Amount  \
2                   RM0098     Always on Time          6194007.0   
3                   RM0231     Always on Time         11159729.0   
6                   RM0213     Always on Time          7678817.0   
12                  RM0341     Always on Time         19829940.0   
16                  RM0378     Always on Time          6863821.0   

    Number_of_Loans  
2               2.0  
3               3.0  
6               2.0  
12              6.0  
16              2.0